In [20]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn 
import torch.optim as optim



In [24]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # 3 32 32 -> 32 16 16
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
        )
        # 32 16 16 -> 64 8 8
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.fc = torch.nn.Linear(8 * 8 * 64, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)  # 가중치 초기화 방식

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)  # flatten
        out = self.fc(out)
        return out


def main():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    torch.manual_seed(777)
    if device == "cuda":
        print("cuda avaliable")
        torch.cuda.manual_seed_all(777)

    # transform.Compose: data를 원하는 형태로 가공
    # normalize: (value-mean)/std, 3ch(rgb)라 3개, 0~1을 -1~1로 바꿈, transforms.Normalize((mean1, mean2, mean3), (std1, std2, std3)),
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    batch_size = 256
    num_workers = 4  # cpu 사용량, 스레드의 절반 정도면 무난하다.
    learning_rate = 0.001
    training_epochs = 15

    trainset = torchvision.datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers, drop_last=True
    )

    testset = torchvision.datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)
    testloader = trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=batch_size, shuffle=False, num_workers=num_workers, drop_last=True
    )
    classes = ("plane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck")

    model = CNN().to(device)
    criterion = torch.nn.CrossEntropyLoss().to(device)  # softmax 포함되어 있음
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    total_batch = len(trainloader)
    print(f"총 배치의 수: {total_batch}")

    for epoch in range(training_epochs):
        avg_cost = 0

        for X, Y in trainloader:  # 미니 배치 단위로 꺼내온다, X = 미니배치, Y = label
            # image is already size of 28x28, no reshape
            # label is not one-hot encoded
            X = X.to(device)
            Y = Y.to(device)

            optimizer.zero_grad()
            hypothesis = model(X)
            cost = criterion(hypothesis, Y)
            cost.backward()
            optimizer.step()

            avg_cost += cost / total_batch

        with torch.no_grad():
            avg_accuracy = 0
            count = 0
            for x, y in testloader:
                count += 1
                x_test = x.to(device)
                y_test = y.to(device)
                prediction = model(x_test)
                correct_prediction = torch.argmax(prediction, 1) == y_test
                avg_accuracy += correct_prediction.float().mean()
            avg_accuracy = avg_accuracy/count
            print(f"[Epoch: {epoch+1:>4}] cost = {avg_cost:>.9} test_accuracy = {avg_accuracy:>.9}")

    # test


if __name__ == "__main__":
    # 멀티태스킹 오류 안나려면 이렇게 실행해야 한다
    main()


cuda avaliable
Files already downloaded and verified
Files already downloaded and verified
총 배치의 수: 195
[Epoch:    1] cost = 1.56508565 test_accuracy = 0.548998415
[Epoch:    2] cost = 1.19256425 test_accuracy = 0.611758828
[Epoch:    3] cost = 1.05846155 test_accuracy = 0.645232379
[Epoch:    4] cost = 0.969720602 test_accuracy = 0.674839735
[Epoch:    5] cost = 0.905679762 test_accuracy = 0.697455943
[Epoch:    6] cost = 0.854334891 test_accuracy = 0.713401437
[Epoch:    7] cost = 0.812100172 test_accuracy = 0.727183521
[Epoch:    8] cost = 0.776243687 test_accuracy = 0.739282846
[Epoch:    9] cost = 0.74466908 test_accuracy = 0.751422286
[Epoch:   10] cost = 0.716494799 test_accuracy = 0.762099385
[Epoch:   11] cost = 0.690256178 test_accuracy = 0.770673096
[Epoch:   12] cost = 0.666142941 test_accuracy = 0.778645873
[Epoch:   13] cost = 0.643589854 test_accuracy = 0.784915864
[Epoch:   14] cost = 0.622709394 test_accuracy = 0.788721979
[Epoch:   15] cost = 0.603397191 test_accuracy

In [13]:
input=torch.randn(20,16,50,100)
print(input[0][0][0].size())
m=torch.nn.Conv2d(16,33,3,stride=2)
output=m(input);
output.size()

torch.Size([100])


torch.Size([20, 33, 24, 49])